<a href="https://colab.research.google.com/github/sourcecode369/deep-nlp/blob/master/text%20generation/Text_Generation_with_an_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, print_function, unicode_literals, absolute_import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
%tensorflow_version 2.x
import tensorflow as tf
import os
import time

TensorFlow 2.x selected.


In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [0]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [0]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
vocab = sorted(set(text))
print('{} unique characters.'.format(len(vocab)))

65 unique characters.


In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print("{")
for char, _ in zip(char2idx, range(20)):
  print(' {:4s}: {:3d}, '.format(repr(char), char2idx[char]))
print('   ...\n')

{
 '\n':   0, 
 ' ' :   1, 
 '!' :   2, 
 '$' :   3, 
 '&' :   4, 
 "'" :   5, 
 ',' :   6, 
 '-' :   7, 
 '.' :   8, 
 '3' :   9, 
 ':' :  10, 
 ';' :  11, 
 '?' :  12, 
 'A' :  13, 
 'B' :  14, 
 'C' :  15, 
 'D' :  16, 
 'E' :  17, 
 'F' :  18, 
 'G' :  19, 
   ...



In [0]:
print(f'{repr(text[:15])} ------ characters mapped to int ------------> {text_as_int[:15]}')

'First Citizen:\n' ------ characters mapped to int ------------> [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [0]:
seq_length = 100

examples_per_epoch = len(text)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

F
i
r
s
t
 
C
i
t
i


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(''.join(idx2char[item.numpy()]))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki
ll him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be d
one: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citi


In [0]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in dataset.take(1):
  print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ',repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step: {:4d}".format(i))
  print(" input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print(" target: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step:    0
 input: 18 ('F')
 target: 47 ('i')
Step:    1
 input: 47 ('i')
 target: 56 ('r')
Step:    2
 input: 56 ('r')
 target: 57 ('s')
Step:    3
 input: 57 ('s')
 target: 58 ('t')
Step:    4
 input: 58 ('t')
 target: 1 (' ')


In [0]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset.element_spec

(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None),
 TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
                               tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                               tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                              #  tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')),
                              #  tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')),
                               tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 10,319,169
Trainable params: 10,319,169
Non-trainable params: 0
_________________________________________________________________


In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# batch_size, sequence_length, vocab_size")

(64, 100, 65) # batch_size, sequence_length, vocab_size


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

**Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.**c

Try it for the first example in the batch:

In [0]:
print('Input: \n', repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n",repr("".join(idx2char[sampled_indices])))

Input: 
 'it.\n\nSecond Gentleman:\nWhat, pray you, became of Antigonus, that carried\nhence the child?\n\nThird Gen'

Next Char Predictions: 
 "iFyl.o\ngEyUM:T';V fVeIbmhZQL\nasQgpV mn'pGfK!CNfmgUAcSJF$3C?xMmi:t,mred\nm$TaHuNpAsnUp?mifaGpGbhDs'zOX"


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Predictions shape: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
print("Scalar loss: ", example_batch_loss.numpy().mean())

Predictions shape:  (64, 100, 65) # (batch_size, sequence_length, vocab_size)
Scalar loss:  4.1737604


In [0]:
model.compile(optimizer="adam", loss=loss)

In [0]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS = 30

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/30
172/172 [==============================] - 19s 109ms/step - loss: 2.6462
Epoch 2/30
172/172 [==============================] - 17s 99ms/step - loss: 1.8416
Epoch 3/30
172/172 [==============================] - 17s 100ms/step - loss: 1.5764
Epoch 4/30
172/172 [==============================] - 17s 99ms/step - loss: 1.4541
Epoch 5/30
172/172 [==============================] - 17s 99ms/step - loss: 1.3831
Epoch 6/30
172/172 [==============================] - 17s 99ms/step - loss: 1.3295
Epoch 7/30
172/172 [==============================] - 17s 100ms/step - loss: 1.2830
Epoch 8/30
172/172 [==============================] - 17s 99ms/step - loss: 1.2381
Epoch 9/30
172/172 [==============================] - 17s 100ms/step - loss: 1.1937
Epoch 10/30
172/172 [==============================] - 17s 99ms/step - loss: 1.1446
Epoch 11/30
172/172 [==============================] - 17s 100ms/step - loss: 1.0927
Epoch 12/30
172/172 [==============================] - 17s 1

In [0]:
# https://github.com/Skuldur/Classical-Piano-Composer

tf.train.latest_checkpoint(checkpoint_dir=checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [0]:
model = build_model(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir=checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 10,319,169
Trainable params: 10,319,169
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  num_generate = 10000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 1.0
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string='RAGINI: '))

RAGINI: I am out of saint, Vomand;
What lovely years? and do your person prince
Even witless but the bow of Times,
Reguiment easher write of sense.
That I would r for't hat slept no more: I am like no season'd by the interrupted waters,
To draw with idle spiders' services
You do limbund waves by sulmnds:
But little thing to fall process of my tongue.

LADY ANNE:
I would I knew thy heart's great beauty in her absence
But dull all ready at the threshold
Will have just out.

CORIOLANUS:
Tush, tush!

MENENIUS:
What would you have, fright im my true love's hand,
Laugh and dispersed by tears in Romeo!

ROMEO:
I stretch it out fetch my lips, Arch, safely did swill be my will
I keep the law,
Now purses, Seeking and harrow in heaven blest in no kind with blood,
Were but a foil of York; and I can leave you to your forwardish'd, put up the squay of holy oath?

DUKE VINCENTIO:
He cannot be most welcome! A mabest, I have death
stabb'd my Edward's grave, lords and watchful gentle king,
And adventure